<a href="https://colab.research.google.com/github/kavyajeetbora/EAST/blob/master/EAST_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Imports

[**Tasks - Incidental Scene Text**](https://rrc.cvc.uab.es/?ch=4&com=tasks)

In [0]:
import os
import time
from PIL import Image
from torch.utils.model_zoo import load_url
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import shutil
from google.colab import files
from zipfile import ZipFile

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [9]:
%%time
%cd ..
directory = '/content/cloned-repo'
if os.path.exists(directory):
  shutil.rmtree(directory)

!git clone -l -s git://github.com/kavyajeetbora/EAST.git /content/cloned-repo
%cd /content/cloned-repo
!ls

/
Cloning into '/content/cloned-repo'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 237 (delta 20), reused 0 (delta 0), pack-reused 199
Receiving objects: 100% (237/237), 18.04 MiB | 8.88 MiB/s, done.
Resolving deltas: 100% (135/135), done.
/content/cloned-repo
dataset.py		     EAST_resnet.py		 model.py
detect.py		     EAST_resnet_training.ipynb  model_resnet.py
dsiplay_ground_truth.ipynb   EAST_training.ipynb	 train.py
EAST_detection.ipynb	     EAST_trial_run.ipynb
EAST_resnet_detection.ipynb  loss.py
CPU times: user 81.4 ms, sys: 12.5 ms, total: 93.9 ms
Wall time: 11 s


In [0]:
from dataset import *

In [0]:
## unzipping the files
from zipfile import ZipFile

def unzip_files(file,output_dir):
  with ZipFile(file, 'r') as zipObj:
    # Extract all the contents of zip file in current directory
    zipObj.extractall(output_dir)
  print('Extracted to',output_dir)

def make_directory(directory):
  if os.path.isdir(directory):
    shutil.rmtree(directory)
  
  os.mkdir(directory)
  print('Created a new directory')

training_data_zip = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Training Set/Random 5000.zip'
test_data_zip = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Test Set/real_Image_dataset_Detection.zip'

In [21]:
# make directories
make_directory('Training Set')
make_directory('Test Set')

Created a new directory
Created a new directory


In [22]:
%%time
unzip_files(training_data_zip,'Training Set')

Extracted to Training Set
CPU times: user 3.35 s, sys: 1.14 s, total: 4.48 s
Wall time: 6.13 s


In [23]:
%%time
unzip_files(test_data_zip,'Test Set')

Extracted to Test Set
CPU times: user 453 ms, sys: 83.3 ms, total: 536 ms
Wall time: 1.76 s


In [30]:
image_folder, label_folder = 'Training Set/Images', 'Training Set/Annotations'
print(len(os.listdir(image_folder)))

12500


In [31]:
print(len(os.listdir(label_folder)))

12500


In [32]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [33]:
%%time
from model import EAST
from loss import Loss

model = EAST()
pretrained_path = '/content/drive/My Drive/east_vgg16.pth'
state_dict = torch.load(pretrained_path)
model.load_state_dict(state_dict)
criterion = Loss()

CPU times: user 1.87 s, sys: 241 ms, total: 2.11 s
Wall time: 2.06 s


## Training Full Batch 

In [0]:
torch.cuda.empty_cache()

In [0]:
def train(model, train_img_path, train_gt_path, batch_size, lr, epoch_iter, num_workers=4, interval=1):
    file_num = len(os.listdir(train_img_path))
    print(file_num)
    trainset = custom_dataset(train_img_path, train_gt_path)
    train_loader = data.DataLoader(trainset, batch_size=batch_size, \
                                   shuffle=True, num_workers=num_workers, drop_last=True)
	
    criterion = Loss()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.8)
    loss_curve = []
    
    for epoch in range(epoch_iter):
      model.train()
      scheduler.step()
      epoch_loss = 0
      epoch_time = time.time()
      for i, (img, gt_score, gt_geo, ignored_map) in enumerate(train_loader):
        start_time = time.time()
        img, gt_score, gt_geo, ignored_map = img.to(device), gt_score.to(device), gt_geo.to(device), ignored_map.to(device)
        pred_score, pred_geo = model(img)
        loss = criterion(gt_score, pred_score, gt_geo, pred_geo, ignored_map)

        epoch_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print('Epoch is [{}/{}], mini-batch is [{}/{}], time consumption is {:.8f}, batch_loss is {:.8f}'.format(\
                  epoch+1, epoch_iter, i+1, int(file_num/batch_size), time.time()-start_time, loss.item()))
        #break
      print('epoch_loss is {:.8f}, epoch_time is {:.8f}'.format(epoch_loss/int(file_num/batch_size), time.time()-epoch_time))
      print(time.asctime(time.localtime(time.time())))
      print('='*50)
      
      if (epoch + 1) % interval == 0:
        state_dict = model.state_dict()
        # replace the weight file
        filename = '/content/drive/My Drive/east_vgg16.pth'.format(epoch)
        if os.path.exists(filename):
          os.unlink(filename)
        torch.save(state_dict, '/content/drive/My Drive/east_vgg16.pth'.format(epoch+1))
       
      ## plotting the loss curve
      loss_curve.append(epoch_loss/int(file_num/batch_size))

    return loss_curve

In [0]:
%%time
loss_curve = train(model, image_folder, label_folder, batch_size=4, lr=1e-5, epoch_iter=30, interval = 1)

12500


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


classify loss is 0.14388162, angle loss is 0.00456179, iou loss is 0.19857121
Epoch is [1/30], mini-batch is [1/3125], time consumption is 0.25949860, batch_loss is 0.38807067
classify loss is 0.36379367, angle loss is 0.01565465, iou loss is 0.67975569
Epoch is [1/30], mini-batch is [2/3125], time consumption is 0.19941449, batch_loss is 1.20009589
classify loss is 0.16569024, angle loss is 0.00317722, iou loss is 0.25280610
Epoch is [1/30], mini-batch is [3/3125], time consumption is 0.18951964, batch_loss is 0.45026851
classify loss is 0.22788912, angle loss is 0.00213697, iou loss is 0.37376446
Epoch is [1/30], mini-batch is [4/3125], time consumption is 0.15294003, batch_loss is 0.62302327
classify loss is 0.29029971, angle loss is 0.00736164, iou loss is 0.42469031
Epoch is [1/30], mini-batch is [5/3125], time consumption is 0.15140510, batch_loss is 0.78860641
classify loss is 0.17409390, angle loss is 0.00471116, iou loss is 0.25351188
Epoch is [1/30], mini-batch is [6/3125], t

In [0]:
plt.plot(loss_curve)

1. [**Adam and AdamW algorithms**](https://www.fast.ai/2018/07/02/adam-weight-decay/)

2. [**The optimal learning rate during fine-tuning of an artificial neural network**](https://www.mikulskibartosz.name/the-optimal-learning-rate-during-fine-tuning-of-an-artificial-neural-network/)

In [6]:
0.7**5 * 1e-5

1.6806999999999996e-06